In [71]:
import pandas as pd

In [72]:
fh = '../files/tickets-gen-all.csv'

In [73]:
df = pd.read_csv(fh, index_col=0, parse_dates=['created', 'opened_at', 'updated_on', 'resolved'])

In [74]:
df.shape

(75724, 22)

In [75]:
df.columns

Index(['created', 'caller', 'opened_at', 'opened_by', 'priority', 'state',
       'category', 'subcategory', 'assignment_group', 'bpl_location',
       'nyp_locaiton', 'resolved', 'assigned_to', 'system', 'closed_at',
       'closed_by', 'mat_source', 'reassignment_count', 'reopen_count',
       'resolved_by', 'updated_on', 'updated_by'],
      dtype='object')

In [76]:
df['caller'].describe()

count          75707
unique          1931
top       Diane Chin
freq             971
Name: caller, dtype: object

In [77]:
df.dtypes

created               datetime64[ns]
caller                        object
opened_at             datetime64[ns]
opened_by                     object
priority                      object
state                         object
category                      object
subcategory                   object
assignment_group              object
bpl_location                  object
nyp_locaiton                  object
resolved              datetime64[ns]
assigned_to                   object
system                        object
closed_at                     object
closed_by                     object
mat_source                    object
reassignment_count             int64
reopen_count                   int64
resolved_by                   object
updated_on            datetime64[ns]
updated_by                    object
dtype: object

In [78]:
df['priority'].unique()

array(['3 - Moderate', '1 - Critical', '5 - Planning', '4 - Low', nan,
       '2 - High'], dtype=object)

In [79]:
df['state'].unique()

array(['Closed', 'Active', 'Awaiting User Info', 'Awaiting Vendor', 'New',
       'Resolved'], dtype=object)

In [80]:
df['category'].unique()

array(['Cataloging', 'Acquisitions', 'Selection', 'Logistics',
       'Offsite - ReCAP', 'Collections Processing', nan], dtype=object)

In [81]:
df['subcategory'].unique()

array(['Item attached to wrong bib record', 'Item not linked', 'Other',
       'Call number error', 'Cataloging error', 'Packing list error',
       'Materials not processed with labels, etc.',
       'Holds fulfillment--NYPL only', 'Item record problem',
       'Volume record needed', 'Duplicate records',
       'Supplies: Circ bins, discard bins, bin ties, tubs, etc.',
       'Call number missing', 'Location code error',
       'Shipment delivery error', 'Status of request',
       'Pickup or delivery request', 'Spine labels', 'Damaged material',
       'Invoice payment', 'Materials received without paperwork',
       'Author and submission inquiries',
       'Barcodes for circulating materials', 'Collection HQ',
       'Online Catalog', 'Gift books or Donations--NYPL only',
       'Barcoding', 'OCLC Holdings', 'Delivery issue or error',
       'New Item/Title Request', 'Electronic resource',
       'Damaged material / missing pieces', 'BiblioCommons',
       'Leased items', 'Book ja

In [82]:
df['reassignment_count'].describe()

count    75724.000000
mean         0.110916
std          0.364769
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          8.000000
Name: reassignment_count, dtype: float64

In [83]:
adf_out = pd.DataFrame(columns=['dept', 'year', 'tickets'])

In [84]:
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for agroup, adf in df.groupby('category'):
    d = dict()
    for y, ydf in adf.groupby(adf['created'].map(lambda x: x.year)):
        d[y]={'dept': agroup, 'year': y, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
            adf_out = adf_out.append(d[y], ignore_index=True)
        else:
            adf_out = adf_out.append({'dept': agroup, 'year': y, 'tickets': 0}, ignore_index=True)         

In [85]:
adf_out.head()

,dept,year,tickets
0,Acquisitions,2013,110
1,Acquisitions,2014,139
2,Acquisitions,2015,185
3,Acquisitions,2016,290
4,Acquisitions,2017,325


In [86]:
adf_out.to_csv('../data-display/tickets-per-dept-yearly.csv', index=False)

In [87]:
# open, awaiting tickets per department
odf = df[(df['state']!='Closed')&(df['state']!='Resolved')]

In [88]:
odf.shape

(3409, 22)

In [89]:
odf_out = pd.DataFrame(columns=['dept', 'year', 'tickets'])
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for agroup, adf in odf.groupby('category'):
    d = dict()
    for y, ydf in adf.groupby(adf['created'].map(lambda x: x.year)):
        d[y]={'dept': agroup, 'year': y, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
            odf_out = odf_out.append(d[y], ignore_index=True)
        else:
            odf_out = odf_out.append({'dept': agroup, 'year': y, 'tickets': 0}, ignore_index=True)

In [90]:
odf_out.head()

,dept,year,tickets
0,Acquisitions,2013,13
1,Acquisitions,2014,20
2,Acquisitions,2015,32
3,Acquisitions,2016,53
4,Acquisitions,2017,65


In [91]:
odf_out.to_csv('../data-display/unresolved-per-dept-yearly.csv', index=False)

In [92]:
# filter out tickets that did not end up in cataloging
cdf = df[df['assignment_group']=='BKOPS CAT']

In [93]:
cdf.shape

(14470, 22)

In [94]:
cdf['assignment_group'].unique()

array(['BKOPS CAT'], dtype=object)

## reassignment count

In [166]:
reas_df = cdf.copy(deep=True)

In [168]:
reas_df['reassignment_count'].value_counts()

0    13608
1      658
2      181
3       15
4        7
5        1
Name: reassignment_count, dtype: int64

In [ ]:
reas_df_out = pd.DataFrame(columns=['reassignment #', 'library', 'state', 'tickets', 'created_date', 'resolved_date'])
for state, rdf in reas_df.groupby('reassigment_count'):
    d = dict()
    for lib, lib_rdf in rdf.groupby('system'):
        pass
    

## State

In [95]:
cdf['category'].unique()
states = ['Closed', 'Resolved', 'Awaiting User Info', 'Awaiting Vendor', 'Active', 'New']

In [96]:
sdf_out = pd.DataFrame(columns=['year', 'state', 'tickets', 'stack_order'])
for state, sdf in cdf.groupby('state'):
    d = dict()
    for y, ydf in sdf.groupby(sdf['created'].map(lambda x: x.year)):
        d[y] = {'year': y, 'state': state, 'tickets': ydf.shape[0], 'stack_order': states.index(state)}
    for y in years:
        if y in d.keys():
            sdf_out = sdf_out.append(d[y], ignore_index=True)
        else:
            sdf_out = sdf_out.append({'year': y, 'state': state, 'tickets': 0, 'stack_order': states.index(state)}, ignore_index=True)

In [97]:
sdf_out.shape

(48, 4)

In [98]:
sdf_out.to_csv('../data-display/cat-tickets-per-state-yearly.csv', index=False)

In [99]:
# misassigned tickets in CAT
vcdf = df[(df['category'] == 'Cataloging')&(df['assignment_group'] != 'BKOPS CAT')|(df['assignment_group']=='BKOPS CAT')]

In [100]:
vcdf.shape[0]

15070

In [101]:
vcdf['valid'] = vcdf.apply(lambda row: 'correct' if row.assignment_group == 'BKOPS CAT' else 'misassigned', axis=1)

c:\users\tomaszkalata\envs\servicenowanalysisenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [102]:
vcdf['valid'].value_counts()

correct        14470
misassigned      600
Name: valid, dtype: int64

In [153]:
misassigned = vcdf[vcdf['assignment_group'] != 'BKOPS CAT']

In [154]:
misassigned.shape[0]

600

In [161]:
misassigned['assignment_group'].value_counts().head()

BKOPS CPRC     319
BKOPS ACQ      158
BKOPS SEL       83
BKOPS RECAP     17
BKOPS ITIL       5
Name: assignment_group, dtype: int64

In [162]:
misassigned['assignment_group'].value_counts().to_csv('../data-display/cat-misassigned-source.csv')

In [103]:
vcdf_out = pd.DataFrame(columns=['year', 'valid', 'tickets'])

In [104]:
for valid, vdf in vcdf.groupby('valid'):
    d = dict()
    for y, ydf in vdf.groupby(vdf['created'].map(lambda x: x.year)):
        d[y] = {'year': y, 'valid': valid, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
            vcdf_out = vcdf_out.append(d[y], ignore_index=True)
        else:
            vcdf_out = vcdf_out.append({'year': y, 'valid': valid, 'tickets': 0}, ignore_index=True)

In [105]:
vcdf_out.head()

,year,valid,tickets
0,2013,correct,1201
1,2014,correct,2018
2,2015,correct,2678
3,2016,correct,2471
4,2017,correct,2144


In [106]:
vcdf_out.to_csv('../data-display/cat-tickets-misassigned.csv', index=False)

## Priority

In [165]:
cdf['priority'].value_counts().to_csv('../data-display/cat-tickets-priority.csv')

## Average open and resolved per work day

In [107]:
# days during the analysis period
rng = pd.date_range(start='5/7/2013', end='2/15/2020')
rng.shape

(2476,)

In [108]:
days_df = pd.DataFrame(index=rng)
days_df['weekday'] = days_df.index.map(lambda x: x.weekday())
days_df.head()

,weekday
2013-05-07,1
2013-05-08,2
2013-05-09,3
2013-05-10,4
2013-05-11,5


In [109]:
# group tickets by created date
dcdf = cdf.copy(deep=True)
dcdf['created_date'] = dcdf['created'].map(lambda x: x.date())

In [110]:
gdcdf = dcdf.groupby('created_date')

In [111]:
tickets_by_date = pd.DataFrame(gdcdf.size(), columns=['tickets'])

In [112]:
tickets_by_date.head()

,tickets
created_date,
2013-05-07,2
2013-05-08,4
2013-05-09,8
2013-05-11,1
2013-05-13,8


In [113]:
tickets_by_date_merged = pd.merge(days_df, tickets_by_date, how='left', left_index=True, right_index=True, sort=True)
tickets_by_date_merged['tickets'] = tickets_by_date_merged['tickets'].fillna(0)

In [114]:
tickets_by_date_merged.head(10)

,weekday,tickets
2013-05-07,1,2.0
2013-05-08,2,4.0
2013-05-09,3,8.0
2013-05-10,4,0.0
2013-05-11,5,1.0
2013-05-12,6,0.0
2013-05-13,0,8.0
2013-05-14,1,0.0
2013-05-15,2,9.0
2013-05-16,3,4.0


In [115]:
tickets_by_date_merged.describe()

,weekday,tickets
count,2476.000000,2476.000000
mean,3.000000,5.844103
std,1.999394,6.113401
min,0.000000,0.000000
25%,1.000000,2.000000
50%,3.000000,5.000000
75%,5.000000,8.000000
max,6.000000,133.000000


In [116]:
d = tickets_by_date_merged.groupby('weekday')['tickets'].describe()
d.to_csv('../data-display/cat-tickets-by-day-described.csv')

In [117]:
tickets_by_date_merged.to_csv('../data-display/cat-tickets-by-day.csv')

In [118]:
tickets_by_date_merged.idxmax()

weekday   2013-05-12
tickets   2014-08-19
dtype: datetime64[ns]

In [119]:
# show BPL & NYPL tickets separately
dcdf.columns

Index(['created', 'caller', 'opened_at', 'opened_by', 'priority', 'state',
       'category', 'subcategory', 'assignment_group', 'bpl_location',
       'nyp_locaiton', 'resolved', 'assigned_to', 'system', 'closed_at',
       'closed_by', 'mat_source', 'reassignment_count', 'reopen_count',
       'resolved_by', 'updated_on', 'updated_by', 'created_date'],
      dtype='object')

In [120]:
dcdf['system'].unique()

array(['NYPL Circulating', 'NYPL Research', 'BPL Circulating'],
      dtype=object)

In [121]:
group_by_system = dcdf.groupby('system')

In [122]:
for lib, lddf in group_by_system:
    grouped_lddf = lddf.groupby('created_date')
    lib_tickets_by_date = pd.DataFrame(grouped_lddf.size(), columns=['tickets'])
    lib_tickets_by_date_merged = pd.merge(days_df, lib_tickets_by_date, how='left', left_index=True, right_index=True, sort=True)
    lib_tickets_by_date_merged['tickets'] = lib_tickets_by_date_merged['tickets'].fillna(0)
    lib_tickets_by_date_merged.to_csv(f'../data-display/{lib}-tickets-by-date.csv')
    lib_tickets_by_date_merged.groupby('weekday')['tickets'].describe().to_csv(f'../data-display/{lib}-tickets-by-day-described.csv')

In [123]:
# resolved tickets
rcdf = cdf[cdf['resolved'].notnull()].copy(deep=True)
rcdf.shape[0]

12004

In [124]:
rcdf['resolved'].head()

number
BKOP0000588   2013-05-10 14:59:35
BKOP0000589   2013-05-10 14:53:38
BKOP0000597   2013-05-28 10:23:04
BKOP0000598   2013-05-28 10:23:37
BKOP0000599   2013-05-09 16:24:58
Name: resolved, dtype: datetime64[ns]

In [125]:
rcdf['created_date'] = rcdf['created'].map(lambda x: x.date())
rcdf['resolved_date'] = rcdf['resolved'].map(lambda x: x.date())

In [126]:
rcdf.columns

Index(['created', 'caller', 'opened_at', 'opened_by', 'priority', 'state',
       'category', 'subcategory', 'assignment_group', 'bpl_location',
       'nyp_locaiton', 'resolved', 'assigned_to', 'system', 'closed_at',
       'closed_by', 'mat_source', 'reassignment_count', 'reopen_count',
       'resolved_by', 'updated_on', 'updated_by', 'created_date',
       'resolved_date'],
      dtype='object')

In [138]:
grouped_rcdf = rcdf.groupby('resolved_date')

In [139]:
resolved_by_date = pd.DataFrame(grouped_rcdf.size(), columns=['resolved tickets'])
resolved_by_date.head()

,resolved tickets
resolved_date,
2013-05-08,1
2013-05-09,1
2013-05-10,2
2013-05-13,5
2013-05-15,2


In [140]:
resolved_by_date_merged = pd.merge(days_df, resolved_by_date, how='left', left_index=True, right_index=True, sort=True)
resolved_by_date_merged['resolved tickets'] = resolved_by_date_merged['resolved tickets'].fillna(0)

In [141]:
resolved_by_date_merged.shape[0]

2476

In [142]:
resolved_by_date_merged.describe()

,weekday,resolved tickets
count,2476.000000,2476.000000
mean,3.000000,4.848142
std,1.999394,6.591360
min,0.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,3.000000
75%,5.000000,7.000000
max,6.000000,134.000000


In [143]:
resolved_by_date_merged.idxmax()

weekday            2013-05-12
resolved tickets   2014-08-27
dtype: datetime64[ns]

In [144]:
rd = resolved_by_date_merged.groupby('weekday')['resolved tickets'].describe()
rd.to_csv('../data-display/cat-resolved-by-day-described.csv')

In [145]:
# NYPL vs BPL
for lib, lrdf in rcdf.groupby('system'):
    grouped_lrdf = lrdf.groupby('resolved_date')
    glrdf = pd.DataFrame(grouped_lrdf.size(), columns=['resolved tickets'])
    lib_resolved_by_date_merged = pd.merge(days_df, glrdf, how='left', left_index=True, right_index=True, sort=True)
    lib_resolved_by_date_merged['resolved tickets'] = lib_resolved_by_date_merged['resolved tickets'].fillna(0)
    lib_resolved_by_date_merged.to_csv(f'../data-display/{lib}-resolved-by-date.csv')
    lib_resolved_by_date_merged.groupby('weekday')['resolved tickets'].describe().to_csv(f'../data-display/{lib}-resolved-by-day-described.csv')
